In [1]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
from fastai.vision.gan import *
from PIL import Image

import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import pdb

# Encoder / Decoder Module

In [ ]:
class MultiUnet(nn.Module):
    def __init__(self, arch:Callable, pretrained:bool=True, cut=None):
        super().__init__()
        
        self.relu = relu(leaky=None)
        
        self.bodyA = create_body(arch, pretrained, cut=-3)
        self.bodyB = create_body(arch, pretrained, cut=-3)
        
        self.sfs_szs = model_sizes(self.bodyA, size=(224,224))
        self.sfs_idxs = list(reversed(_get_sfs_idxs(self.sfs_szs)))
        self.sfsA = hook_outputs([self.bodyA[i] for i in self.sfs_idxs])
        x = dummy_eval(self.bodyA, (224, 224)).detach()
        
        self.sfsB = hook_outputs([self.bodyB[i] for i in self.sfs_idxs])
        x = dummy_eval(self.bodyB, (224, 224)).detach()
        
        unet_blocksA = []
        x = torch.tensor([])
        x = x.new_full((1, 512, 7, 7), 0)
        up_in_c = []
        x_in_c = []
        for i,idx in enumerate(self.sfs_idxs):
            up_in_c.append(int(x.shape[1]))
            x_in_c.append(int(self.sfs_szs[idx][1]))
            not_final = i!=len(self.sfs_idxs)-1
            block = UnetBlock(int(x.shape[1]), int(self.sfs_szs[idx][1]), self.sfsA[i], final_div=not_final, blur=False, self_attention=False).eval()
            x = block(x)
        
        #DecoderA
        self.decoderA = Decoder(256)
        self.UpBlockA1 = UpBlock(256, 128)
        self.UpBlockA2 = UpBlock(128, 64)
        self.UpBlockA3 = UpBlock(64, 64)
        self.finalDecoderA = nn.Sequential(PixelShuffle_ICNR(64), conv_layer(64, 3))
        self.ResA = ResBlocks(4, 256, 'in', 'relu', padding=1)
        
        #DecoderB
        self.UpBlockB1 = UpBlock(256, 128)
        self.UpBlockB2 = UpBlock(128, 64)
        self.UpBlockB3 = UpBlock(64, 64)
        self.ResB = ResBlocks(4, 256, 'in', 'relu', padding=1)
        self.finalDecoderB = nn.Sequential(PixelShuffle_ICNR(64), conv_layer(64, 3))

        #Shared Layers
        self.sharedEncoderLayer = conv_layer(256, 512, stride=2)
        self.middleConv = nn.Sequential(nn.BatchNorm2d(512), nn.ReLU(512), conv_layer(512, 512*2, stride=1), nn.Conv2d(512*2, 512, 3, stride=1))
        self.UpShared = UpBlock(512, 256)

        #Tan layer
        self.tanLayer = nn.Tanh()
    
    
    def EncoderA(self, xb):
        result = self.bodyA(xb)
        return result
    
    def EncoderB(self, xb):
        result = self.bodyB(xb)
        return result
    
    def sharedEncoder(self, xb):
        result = self.sharedEncoderLayer(xb)
        return result
        
    def MiddleConv(self, xb):
        result = self.middleConv(xb)
        return result
    
    def sharedDecoder(self, xb):
        return self.UpShared(xb, None)
    
    def DecoderA(self, xb, body):
        xb = self.ResA(xb)
        xb = self.UpBlockA1(xb, body[0].stored)
        xb = self.UpBlockA2(xb, body[1].stored)
        xb = self.UpBlockA3(xb, body[2].stored)

        return self.finalDecoderA(xb)
    
    def DecoderB(self, xb, body):
        xb = self.ResB(xb)
        xb = self.UpBlockB1(xb, body[0].stored)
        xb = self.UpBlockB2(xb, body[1].stored)
        xb = self.UpBlockB3(xb, body[2].stored)

        return self.finalDecoderB(xb)
    
    def forward(self, a, b, *pred):
        
        #get initial encodings of both
        a,b = self.EncoderA(a), self.EncoderB(b)
        
        #put both through shared encoder and middle conv
        a,b = self.sharedEncoder(a), self.sharedEncoder(b)
        a,b = self.middleConv(a), self.middleConv(b)
        
        #put images through shared decoder
        a,b = self.sharedDecoder(a), self.sharedDecoder(b)
        
        #Get images that are supposed to be
        aToA, bToB = self.DecoderA(a, body=self.sfsA),self.DecoderB(b, body=self.sfsB)

        #Get switched images
        aToB, bToA = self.DecoderB(a, body=self.sfsA), self.DecoderA(b, body=self.sfsB)
    
        allIm = torch.cat((self.tanLayer(aToA), self.tanLayer(bToB), self.tanLayer(aToB), self.tanLayer(bToA)), 0)
        
        return allIm
    